In [12]:
import json

words = []
classes = []
documents = []
ignore_words = ['?', '!', 'bot', ]
data_file = open('../../data/chat/intents.json').read()
intents = json.loads(data_file)
intents

{'intents': [{'tag': 'tag:1',
   'patterns': 'اسمت چیه',
   'responses': ['خوشتیپ محله',
    'به تو چه',
    'روبوکیا',
    'چنگیز',
    'توله خر هستم ☺️🐴',
    'من همنام پسر عمومتم ینی دلک bot گاو ماچ خر',
    'bot پر حرف',
    'اسم ندارم',
    'bot هستم اسکل و روانی محبوب قلب همه هم نیستم',
    'به تو چه',
    'فضولی',
    'با اجازه ابوالفضل bot هستم',
    'کونی هستم',
    'لئوناد داوینچیه',
    'bot 😍😍معنی اسمم الهه ی ماه 🌙🌜',
    'اصغر کونی',
    'اول اینکه سلام دوم اینکه اسمم bot و میتونم باهاتون صحبت کنم و شما هم میتونین بهم حرف یاد بدین \u200c حرفای بد گزارش میشن و پدرتون درمیاد گفته باشم',
    'bot',
    'تربچه خونت کجاست /تو باغچه چند تا بچه داری /به تو چه',
    'نازنین',
    'bot جون😍',
    'تا جونت در آد',
    'bot بلا',
    'bot',
    'چنگیی',
    'بتوچه',
    'bot قهبه',
    'تربچه',
    'ترپچه🗿',
    'ترپچه',
    'bot جون و اسم شما',
    'عباس بو عذار',
    'کیر',
    'روژان',
    'پرویز پرستویی',
    'رضا گلزار با کمال احترام',
    'راک',
    'کثافت تن لش',
    'سامانتا'

In [9]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\me\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\me\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\me\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [13]:

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import random

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

pickle.dump(words,open('../../data/words_v2.pkl','wb'))
pickle.dump(classes,open('../../data/classes_v2.pkl','wb'))

In [ ]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

In [ ]:
train_y

In [ ]:
from keras.optimizer_v2.adam import Adam

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

adamop = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adamop, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=32, verbose=2)
model.save('../../data/chatbot_model_v2.h5', hist)

print("model created")